### Imports & Settings
- Descargar e importar dependencias

In [142]:
# Load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dummy baseline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score, mean_squared_error

# Preprocessing (Pipelines)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modeling: Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Modeling: Random Forest
from sklearn.ensemble import RandomForestClassifier

# Modeling: LightGBM
# Suprimir warnings (LGBM suele ser sucio con las warnings)
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*Unknown parameter.*")
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
import warnings

### Load Data
- Cargar el data set
- Primer vistazo a la distribucion

In [92]:
# Get url from file hosted on GitHub
url = "https://raw.githubusercontent.com/ZocoMacc/car-quirks-ml_InnovaLab25/refs/heads/main/data/cars_data.csv"

# Cargar csv en un DataFrame
df = pd.read_csv(url)

# Explorar la estructura del DataFrame (Sanity Check)
df.info()
df.shape
# df.isna().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    10000 non-null  object 
 1   year                    10000 non-null  int64  
 2   selling_price           10000 non-null  int64  
 3   km_driven               10000 non-null  int64  
 4   fuel                    10000 non-null  int64  
 5   combustible_estimado_l  10000 non-null  float64
 6   seller_type             10000 non-null  int64  
 7   transmission            10000 non-null  int64  
 8   owner                   10000 non-null  int64  
 9   tipo_carroceria         10000 non-null  int64  
 10  potencia_motor_hp       10000 non-null  int64  
 11  nivel_seguridad         10000 non-null  float64
 12  calidad_auto            10000 non-null  object 
 13  score_calidad           10000 non-null  float64
 14  eficiencia_km_l         10000 non-null 

(10000, 15)

In [48]:
# Ver el balance de las clases (84% Media, 10.5% Alta, 5.5% Baja)
df["calidad_auto"].value_counts(normalize=True)

,proportion
calidad_auto,
Media,0.8399
Alta,0.1052
Baja,0.0549


##### Lo que esto nos dice
- Un fuerte desbalanceo de clases
  - "Media" es la mayoria con un 84%
  - "Alta" solo cubre el 10%, y "Baja" solo un 5.5%
- El accuracy de referencia (baseline) es engañosamente alto
  - Un DummyClassifier que siempre predice "Media" deberia de poder predecir 83.99%.
  - Superar este baseline no garantiza que el modelo sea mas precizo detectando "Alta" o "Baja".
- Riesgo de abandono de las clases minoritarias
  - El modelo tiene el riesgo de predecir "Media" la mayoria del tiempo debido al desbalanceo de las clases, lo que podria ignorar "Alta" o "Baja" como opciones.

### Cleaning & Dropping
- Decidir las variables que seran ignoradas por el modelo.
- Si es necesarion, transformar valores en interpretaciones legibles por el modelo o marcarlas con una categoria.

In [93]:
# Duda sobre la variable "score_calidad": Que tanto se correlaciona con "calidad_auto"
# Mapear las categorias con numeros para calcular la correlacion
mapping = {"Baja": 0, "Media": 1, "Alta": 2}
df["calidad_code"] = df["calidad_auto"].map(mapping)

# Calcular la correlacion
corr = df[["score_calidad", "calidad_code"]].corr().iloc[0, 1]

# Imprimir los resultados
print(f"Correlacion entre score_calidad y calidad_auto: {corr:.3f}")

Correlacion entre score_calidad y calidad_auto: 0.740


##### Lo que esto nos dice
- La correlacion entre "score_calidad" y "calidad_auto" no es 1.0, lo que quiere decir que "calidad_auto" no depende al 100% de la variable "score_calidad".
- La variable "score_calidad" va a ser considerada.

#### Extraer marca del nombre
- Usar el nombre y cortarlo de una manera en la que podamos extrar la marca del auto para poder usarlo como variable y ser considerada.

In [50]:
"""
Rifense este rollo porfi
- Extraer el nombre de la marca de cada carro (crar una columna para "brand")
- Investigar si es viable extraer tambien el submodelo del carro (Limited, Sport, AC, ZX, etc)
  - Maybe no nos conviene por que son un chingo pero maybe si si no son tantas quien sabe
  - Si si es viable crear una columna para "submodel"
"""

'\nRifense este rollo porfi\n- Extraer el nombre de la marca de cada carro (crar una columna para "brand")\n- Investigar si es viable extraer tambien el submodelo del carro (Limited, Sport, AC, ZX, etc)\n  - Maybe no nos conviene por que son un chingo pero maybe si si no son tantas quien sabe\n  - Si si es viable crear una columna para "submodel"\n'

#### Convertir columnas de enteros en categorias
- Columnas como "fuel", "seller_type", "transmission", "owner", "tipo_carroceria" tienen asignado un valor entero pero en realidad representan categorias sin orden.
- Convertir estas variables a Pandas' category dtype para mas claridad y eficiencia en la manipulacion de estas variables, tambien es mas seguro hacerlo de esta forma, asi evitamos que estas columnas se interpreten como numeric imputers o scalers.

In [94]:
cat_int_cols = [
    "fuel",
    "seller_type",
    "transmission",
    "owner",
    "tipo_carroceria"
    #"brand"             # caegoria extraida del nombre
]

for col in cat_int_cols:
    df[col] = df[col].astype("category")

#### Droppear columnas innecesarias
- Calcular correlaciones entre las columnas para identificar cual tiene un gran efecto en "calidad_auto"
- Definir X, y para entrenamiento

In [95]:
# Al final se droppea el nombre (ya habiendo extraido la brand)
df = df.drop(columns=["name"])

# Ver correlaciones para identificar leaks
correlations = df.drop(columns=["calidad_auto"]).corrwith(df["calidad_code"])
print("Correlaciones")
print(correlations.sort_values(ascending=False))

# Tirar la columna helper que definimos previemente
df = df.drop(columns=["calidad_code"])

print("\nColumnas que permanecen en el DataFrame:")
print(df.dtypes)  # Checar las columnas que permanecen

# Definir X & y para el training
features_to_drop = ["calidad_auto", "score_calidad"]
X = df.drop(columns=features_to_drop)
y = df["calidad_auto"]    # Estableciendo el target

# Inspeccionar las variables que permanecen
# print("Columnas a considerar:\n", X.columns.tolist())
print("\nFeature dtypes (para X):")
print(X.dtypes)

Correlaciones
calidad_code              1.000000
score_calidad             0.740022
eficiencia_km_l           0.526133
nivel_seguridad           0.497154
year                      0.495617
potencia_motor_hp         0.294303
transmission              0.197374
selling_price             0.103824
fuel                      0.031103
seller_type               0.028950
km_driven                -0.024826
tipo_carroceria          -0.033271
owner                    -0.045147
combustible_estimado_l   -0.107143
dtype: float64

Columnas que permanecen en el DataFrame:
year                         int64
selling_price                int64
km_driven                    int64
fuel                      category
combustible_estimado_l     float64
seller_type               category
transmission              category
owner                     category
tipo_carroceria           category
potencia_motor_hp            int64
nivel_seguridad            float64
calidad_auto                object
score_calidad      

### DummyClassifier & Baseline
- El primer paso es establecer una baseline y asegurarse de que la informacion esta limpia
- El objetivo es confirmar ques este DummyClassifier de como resultado una precision de 83.99%
- Se usara la estrategia de "most_frequent", simplemente el valor que mas se repite sera elegido para la prediccion.
- Este modelo representa el piso (baseline) de qualquier modelo futuro.
- El split para entrenamiento es un 80/20 asegurandose de que cada clase aparece con las misma proporciones.



In [96]:
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Test
# print("Train class proportions:\n", y_train.value_counts(normalize=True))
# print("\nTest class proportions:\n", y_test.value_counts(normalize=True))

# Iniciar DummyClassifier
dummy = DummyClassifier(
    strategy="most_frequent", # Siempre predice la clase mas frecuente ("Media")
    random_state=42
)

dummy.fit(X_train, y_train)     # "Entrena" basado en la clase mas frecuente

# Obtener predicciones
y_pred = dummy.predict(X_test)  # Aplica la regla a cada fila de X_test

# Evaluar accuracy
accuracy = accuracy_score(y_test, y_pred)   # Fraccion de predicciones correctas
print(f"DummyClassifier Accuracy: {accuracy}")

# Visualizar resulatdos de predicciones
print("Classification report:\n")
print(classification_report(
    y_test,
    y_pred,
    digits=4,         # Numero de decimales
    zero_division=0   # Ignorar zero-division errors (debido a strings)
))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

DummyClassifier Accuracy: 0.84
Classification report:

              precision    recall  f1-score   support

        Alta     0.0000    0.0000    0.0000       210
        Baja     0.0000    0.0000    0.0000       110
       Media     0.8400    1.0000    0.9130      1680

    accuracy                         0.8400      2000
   macro avg     0.2800    0.3333    0.3043      2000
weighted avg     0.7056    0.8400    0.7670      2000

Confusion Matrix:
 [[1680    0    0]
 [ 210    0    0]
 [ 110    0    0]]


#### Lo que esto nos dice
- Tenemos un baseline de ~84% accuracy, este es el objetivo a vencer en los proximos modelos.

### Data preprocessing
- Preparar el data set para ser procesado
- Dividir entre variables numericas y categoricas
- Implementar un pipeline para cada tipo de variable
- Usar ColumnTransformer para aplicar los pipelines al DataFrame

#### Identificar columnas en base al dtype

In [97]:
# Datos numericos
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
print("Numeric features:", num_cols)

# Datos categoricos
cat_cols = X.select_dtypes(include=["category"]).columns.tolist()
print("\nCategorical features:", cat_cols)

Numeric features: ['year', 'selling_price', 'km_driven', 'combustible_estimado_l', 'potencia_motor_hp', 'nivel_seguridad', 'eficiencia_km_l']

Categorical features: ['fuel', 'seller_type', 'transmission', 'owner', 'tipo_carroceria']


#### Construir el pipeline numerico

In [98]:
num_pipe = Pipeline([
    # Definiendo tecnicas de defensive programming
    ("imputer", SimpleImputer(strategy="median")), # Imputacion de media
    ("scaler", StandardScaler())                   # Standard scaling
])

- El objetivo de la funcion `SimpleImputer()` es remplazar valores faltantes por el valor definido por su argumento (e.g. median).
- El objetivo de la funcion `StandardScaler()` es que despues de una imputacion, el valor numerico es escalado para tener una media = 0 y standard deviation = 1. De esta forma se evitan posibles errores generados por tener diferentes escalas de numeros en las diferentes columnas.

#### Construir el pipeline categorico

In [99]:
cat_pipe = Pipeline([
    # Definiendo tecnicas de defensive programming
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Imputacion de moda
    ("encoder", OneHotEncoder(handle_unknown="ignore"))    # One-hot encoding
])

- `SimpleImputer()` remplaza cualquier valor faltante por el valor mas frecuente (la moda). Esto preserva la distribucion del data set.
- El objetivo del OneHotEncoder es convertir cada columna categorica en las columnas binarias necesarias para representar cada valor categorico unico.
  - `handle_unkwnown="ignore"` es util para que en caso de que aparezca una nueva categoria "vacia", que esta sea representada por una columna entera de puros ceros.

#### Implementacion de ColumnTransformer para aplicar los pipelines

In [100]:
preprocessor = ColumnTransformer(
    transformers=[
        # Aplicar num_pipe a las columnas numericas
        ("nums", num_pipe, num_cols),
        # Aplicar cat_pipe a columnas categoricas
        ("cats", cat_pipe, cat_cols)
    ],
    remainder="drop" # Tirar cualquier columna no listada
)

- `transformers` es un array de tuples `(name, pipeline, columns)` que ira siendo populado mientras `.fit()` corre
  1. `("nums", num_pipe, num_cols)` toma todas las columnas en `num_cols` y aplica el pipeline `num_pipe` y entraga como output el array numerico transformado.
  2. `("cats", cat_pipe, cat_cols)` toma todas las columnas en `cat_cols` y aplica el pipeline `cat_pipe` y entraga como output el array categorico transformado.

#### Inspeccionar el output transformado

In [101]:
# Testear el preprocessor
preprocessor.fit(X_train)

# Ver la forma del array final
X_train_transformed = preprocessor.transform(X_train)
print("Transformed shape:", X_train_transformed.shape)
#print(len(num_cols))

# Imprimir las columnas finales
feature_names = preprocessor.get_feature_names_out()
print("All output features:\n", feature_names)

Transformed shape: (8000, 27)
All output features:
 ['nums__year' 'nums__selling_price' 'nums__km_driven'
 'nums__combustible_estimado_l' 'nums__potencia_motor_hp'
 'nums__nivel_seguridad' 'nums__eficiencia_km_l' 'cats__fuel_0'
 'cats__fuel_1' 'cats__fuel_2' 'cats__fuel_3' 'cats__fuel_4'
 'cats__seller_type_0' 'cats__seller_type_1' 'cats__seller_type_2'
 'cats__transmission_0' 'cats__transmission_1' 'cats__owner_0'
 'cats__owner_1' 'cats__owner_2' 'cats__owner_3' 'cats__owner_4'
 'cats__tipo_carroceria_1' 'cats__tipo_carroceria_2'
 'cats__tipo_carroceria_3' 'cats__tipo_carroceria_4'
 'cats__tipo_carroceria_5']


##### Lo que esto nos dice
- En total hay 28 columnas ya con los pipelines aplicados (8 numericas y 20 categoricas) - temporal (falta agregar "brand" y talvez "submodel")
- Los prefijos `nums_` y `cats_` nos dicen de cual pipeline proviene cada columna
- Estos nombres nos serviran despues para implementacion de funciones o debugging

### Logistic Regression
- La regresión logística es una técnica de análisis de datos que utiliza las matemáticas para encontrar las relaciones entre dos factores de datos. Luego, utiliza esta relación para predecir el valor de uno de esos factores basándose en el otro. Normalmente, la predicción tiene un número finito de resultados, como un sí o un no.

In [102]:
# Crear un objecto de LogisticRegression
logreg = LogisticRegression(
    multi_class="multinomial",   # Clasificacion multiclase
    solver="saga",               # Algoritmo de optimizacion (soporta L1/L2)
    class_weight="balanced",     # Ajuste de pesos para clases desbalanceadas
    max_iter=1000,               # Numero maximo de iteraciones
    random_state=42
)

# Construir un unico pipeline
lr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", logreg)
])

# Usar 5-fold cross-validation para evaluar el modelo de regresion logistica
cv_macro_f1 = cross_val_score(
    lr_pipeline,    # Pipeline completeo
    X,              # Todas las features
    y,              # Todas las labels
    cv=5,           # 5 folds
    scoring="f1_macro", # macro-averaged F1
    n_jobs=-1       # Usar todos los nucleos del CPU
)

# Imprimir los fold scores y la media
print("Logistic Regression 5-fold CV macro-F1 scores:", cv_macro_f1.round(4))
print("Mean macro-F1:", cv_macro_f1.mean().round(4))

Logistic Regression 5-fold CV macro-F1 scores: [0.947  0.919  0.91   0.8839 0.8849]
Mean macro-F1: 0.909


##### Entendiendo la media Macro-F1
- La evaluacion multiclase Macro-F1 toma el promedio no ponderado de los tres puntajes F1 por clase ("Alta", "Media", "Baja"). Este proceso toma cada clase de forma equivalente aunque la distribucion sea diferente (10.5%, 84%, 5.5%). Usando macro-F1 nos aseguramos de que el modelo no esta eligiendo "Media" en todas las predicciones y realmente esta prediciendo entre las tres clases.
- El metodo de evaluacion 5-fold cross-validation es una manera de estimar que tan bien el pipeline (preprocessor + modelo) se generaliza a datasets no vistos
  - Se divide el dataset completo entre 5 subsets (folds), en este caso cada fold tiene aproximadamente 2000 carros.
  - Al final obtenemos 5 scores, una por cada subset, esto en toeria refleja que tan bien se comportaria el pipeline si fuera un "nuevo" dataset. El promedio de esas 5 scores es lo que llamamos mean macro-F1.

##### Lo que los resulatados del Macro-F1 nos dicen
- Los 5 scores estan arribe de 0.90, lo que quiere decir que el pipeline esta consistentemente obteniendo un score alto en "nueva" informacion.
- La media macro-F1 de 0.9274 es el mejor resumen numérico del "rendimiento esperado" si se entrenó con el 80 % del conjunto de datos y se probó con el 20 % restante. Esto sugiere que, en promedio, el modelo clasifica correctamente casi todos los ejemplos de "Media" y también funciona bien con "Alta" y "Baja".

#### Logistic Regression Training & Testing


In [103]:
# Test 1
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Asegurarse de las columnas presentes en X_train
print("Columns in X_train:", X_train.columns.tolist())

# Fit X_train, y_train (80%) y evaluar en X_test, y_test (20%)
lr_pipeline.fit(X_train, y_train)
y_test_pred = lr_pipeline.predict(X_test)

# Evaluar el test
test_acc = accuracy_score(y_test, y_test_pred)
print(f"Logistic Regression Test Accuracy: {test_acc:.4f}")
print("\nLogistic Regression Test Classification Report:\n")
print(classification_report(
    y_test,
    y_test_pred,
    digits=4,         # Numero de decimales
    zero_division=0   # Ignorar zero-division errors (debido a strings)
))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_test_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

Columns in X_train: ['year', 'selling_price', 'km_driven', 'fuel', 'combustible_estimado_l', 'seller_type', 'transmission', 'owner', 'tipo_carroceria', 'potencia_motor_hp', 'nivel_seguridad', 'eficiencia_km_l']


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Test Accuracy: 0.9615

Logistic Regression Test Classification Report:

              precision    recall  f1-score   support

        Alta     0.8714    1.0000    0.9313       210
        Baja     0.7051    1.0000    0.8271       110
       Media     1.0000    0.9542    0.9765      1680

    accuracy                         0.9615      2000
   macro avg     0.8588    0.9847    0.9116      2000
weighted avg     0.9703    0.9615    0.9636      2000

Confusion Matrix:
 [[1603   31   46]
 [   0  210    0]
 [   0    0  110]]


In [104]:
# Test 2
# 3.3.1 New random split
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=99
)

# 3.3.2 Re‐fit your original pipeline (with score_calidad)
lr_pipeline.fit(X2_train, y2_train)
y2_pred = lr_pipeline.predict(X2_test)

acc2 = accuracy_score(y2_test, y2_pred)
print(f"New split (seed=99) Test Accuracy: {acc2:.4f}")
print("\nNew split Classification Report:\n")
print(classification_report(y2_test, y2_pred, digits=4, zero_division=0))

# Generar confusion matrix
cm = confusion_matrix(y2_test, y2_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


New split (seed=99) Test Accuracy: 0.9670

New split Classification Report:

              precision    recall  f1-score   support

        Alta     0.8678    1.0000    0.9292       210
        Baja     0.7639    1.0000    0.8661       110
       Media     1.0000    0.9607    0.9800      1680

    accuracy                         0.9670      2000
   macro avg     0.8772    0.9869    0.9251      2000
weighted avg     0.9731    0.9670    0.9684      2000

Confusion Matrix:
 [[1614   32   34]
 [   0  210    0]
 [   0    0  110]]


#### Testing Logistic Regression

In [105]:
# Input entry test
lr_pipeline.fit(X_train, y_train)

X.columns.tolist()

new_car_data = {
    "year": [2018],                   # int
    "selling_price": [350000],        # int or float
    "km_driven": [25000],             # int
    "fuel": [1],                      # must match the same category codes you used
    "combustible_estimado_l": [1800.0],  # float
    "seller_type": [0],               # category code
    "transmission": [1],              # category code
    "owner": [0],                     # category code
    "tipo_carroceria": [2],           # category code
    "potencia_motor_hp": [120],       # int
    "nivel_seguridad": [4.5],         # float
    "score_calidad": [5.5],           # float
    "eficiencia_km_l": [18.0]         # float
    # "brand": ["Toyota"]             # string; will be cast to category below
}

new_car_df = pd.DataFrame(new_car_data)

cat_cols = ["fuel", "seller_type", "transmission", "owner", "tipo_carroceria"]
for col in cat_cols:
    new_car_df[col] = new_car_df[col].astype("category")

# new_car_df["brand"] = new_car_df["brand"].astype("category")

print(new_car_df.dtypes)

predicted_label = lr_pipeline.predict(new_car_df)
print("\nPredicted calidad_auto:", predicted_label[0])

predicted_proba = lr_pipeline.predict_proba(new_car_df)
print("Predicted probabilities [Alta, Media, Baja]:", predicted_proba[0])

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


year                         int64
selling_price                int64
km_driven                    int64
fuel                      category
combustible_estimado_l     float64
seller_type               category
transmission              category
owner                     category
tipo_carroceria           category
potencia_motor_hp            int64
nivel_seguridad            float64
score_calidad              float64
eficiencia_km_l            float64
dtype: object

Predicted calidad_auto: Alta
Predicted probabilities [Alta, Media, Baja]: [9.99999983e-01 1.84330860e-30 1.73430678e-08]


### Random Forest
- Random Forest es un algoritmo de aprendizaje supervisado en machine learning que utiliza múltiples árboles de decisión para clasificar o predecir datos. Es un método de conjunto (ensemble method) que mejora la precisión de las predicciones combinando los resultados de varios modelos débiles (árboles de decisión).

In [106]:
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Construir e inicializar un RandomForestClassifer
rf_clf = RandomForestClassifier(
    n_estimators=300,     # Numero de arboles en el bosque
    max_depth=None,       # Profundidad maxima de los arboles
    class_weight="balanced",  # Ajuste de pesos para clases desbalanceadas
    min_samples_split=2,  # Minimo numero de muestras requeridas para dividir
    random_state=42,      # Seed
    n_jobs=-1             # Usar todos los nucleos del CPU
)

# Crear un Pipeline unico que aplica el preprocessor previamente definido
rf_pipeline = Pipeline([
    # Usando el preprocessor que definimos previamente (ColumnTransformer)
    ("preprocessor", preprocessor),
    ("classifier", rf_clf)
])

- `n_estimators=300` construye 300 arbholes de decision
- `class_weight="balanced"` garantiza que las clases minoritarias tengan mayor ponderacion durante las divisiones
- `n_jobs=-1` paraleliza la construccion de arboles en todos los nucleos
- Reutilizamos el mismo preprocesador que gestiona la imputacion de la mediana, el escalado y el One-hot encoding. De esta forma, Random Forest ve la misma matriz numerica de 28 columnnas..

In [107]:
# Verificar las columnas consideradas
print("Features in X:", X.columns.tolist())

# if "score_calidad" in X.columns:
#     X = X.drop(columns=["score_calidad"])
# if "calidad_code" in X.columns:
#     X = X.drop(columns=["calidad_code"])

# Usar 5-fold cross-validation para evaluar el Random Forest
rf_cv_scores = cross_val_score(
    rf_pipeline,
    X,            # full feature set
    y,            # full labels
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

# Imprimir scores del 5-fold test
print("Random Forest 5-fold CV macro-F1 scores:", rf_cv_scores.round(4))
print("Mean macro-F1 (RF):", rf_cv_scores.mean().round(4))

Features in X: ['year', 'selling_price', 'km_driven', 'fuel', 'combustible_estimado_l', 'seller_type', 'transmission', 'owner', 'tipo_carroceria', 'potencia_motor_hp', 'nivel_seguridad', 'eficiencia_km_l']
Random Forest 5-fold CV macro-F1 scores: [0.7811 0.8864 0.8869 0.885  0.8669]
Mean macro-F1 (RF): 0.8613


##### Lo que los resulatados del Macro-F1 nos dicen
- Los 5 scores son relativamente altos, lo que quiere decir que el pipeline esta consistentemente obteniendo un score alto en "nueva" informacion. Sinembargo, parece que el modelo es algo sensible dependeindo de cual 20% es escogido.
- Comparado a al modelo de Regresion Logistica, que tuvo un macro-F1 de 0.909 (ignorando la columna "score_calidad"), el modelo de Random Forest de hecho so comporta un poco peor en el 5-fold test. Esto puede significar que los limites lineales aprendidos por la regresion logistica eran mas precisos al momento de separar "Alta"/"Media"/"Baja" que el Random Forest.

#### Random Forest Training and Testing

In [108]:
# Fit Random Forest pipeline
rf_pipeline.fit(X_train, y_train)

# Predecir en el split de 20%
y_rf_pred = rf_pipeline.predict(X_test)

# Evaluar el modelo
rf_test_acc = accuracy_score(y_test, y_rf_pred)
print(f"Random Forest Test Accuracy: {rf_test_acc:.4f}")

# Classification report
print("\nRandom Forest Classification Report (Test Set):\n")
print(classification_report(
    y_test,
    y_rf_pred,
    digits=4,
    zero_division=0
))

# Confusion matrix
rf_cm = confusion_matrix(y_test, y_rf_pred, labels=["Media", "Alta", "Baja"])
print("\nRandom Forest Confusion Matrix (Test Set):\n", rf_cm)

Random Forest Test Accuracy: 0.9555

Random Forest Classification Report (Test Set):

              precision    recall  f1-score   support

        Alta     0.9448    0.8143    0.8747       210
        Baja     0.9605    0.6636    0.7849       110
       Media     0.9564    0.9923    0.9740      1680

    accuracy                         0.9555      2000
   macro avg     0.9539    0.8234    0.8779      2000
weighted avg     0.9554    0.9555    0.9532      2000


Random Forest Confusion Matrix (Test Set):
 [[1667   10    3]
 [  39  171    0]
 [  37    0   73]]


In [109]:
#Test 2
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=99
)
rf_clf2 = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)
rf_pipeline2 = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", rf_clf2)
])
rf_pipeline2.fit(X2_train, y2_train)
y2_pred = rf_pipeline2.predict(X2_test)
acc2 = accuracy_score(y2_test, y2_pred)
print("New-split Test Accuracy:", acc2)
print(classification_report(y2_test, y2_pred, digits=4, zero_division=0))

# Confusion matrix
rf_cm = confusion_matrix(y2_test, y2_pred, labels=["Media", "Alta", "Baja"])
print("\nRandom Forest Confusion Matrix (Test Set):\n", rf_cm)

New-split Test Accuracy: 0.9545
              precision    recall  f1-score   support

        Alta     0.9389    0.8048    0.8667       210
        Baja     0.9733    0.6636    0.7892       110
       Media     0.9553    0.9923    0.9734      1680

    accuracy                         0.9545      2000
   macro avg     0.9558    0.8202    0.8764      2000
weighted avg     0.9546    0.9545    0.9521      2000


Random Forest Confusion Matrix (Test Set):
 [[1667   11    2]
 [  41  169    0]
 [  37    0   73]]


### Boosted Tree (LightGBM)
- LightGBM es un marco de trabajo de refuerzo de gradiente rápido, distribuido y de alto rendimiento que utiliza un algoritmo de aprendizaje basado en árboles. Está diseñado específicamente para grandes conjuntos de datos y datos de alta dimensión, y ofrece ventajas como mayor velocidad de entrenamiento, menor uso de memoria y mayor precisión en comparación con otros algoritmos de refuerzo. Los árboles reforzados, en general, son un método de aprendizaje conjunto que combina múltiples aprendices débiles (como árboles de decisión) para crear un modelo predictivo sólido.

In [139]:
# Definir un pipeline de LGBM
lgbm_clf = lgb.LGBMClassifier(
    objective="multiclass",
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)
lgbm_pipeline = Pipeline([
    # Usando el preprocessor que definimos previamente (ColumnTransformer)
    ("preprocessor", preprocessor),
    ("classifier", lgbm_clf)
])

# Usar 5-fold cross-validation para evaluar el Random Forest
lgbm_cv_scores = cross_val_score(
    lgbm_pipeline,
    X,
    y,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

# Imprimir scores del 5-fold test
print("LightGBM 5-fold CV macro-F1:", lgbm_cv_scores.round(4))
print("Mean macro-F1 (LGBM):", lgbm_cv_scores.mean().round(4))

LightGBM 5-fold CV macro-F1: [0.941  0.9337 0.94   0.9314 0.9213]
Mean macro-F1 (LGBM): 0.9335


##### Lo que los resulatados del Macro-F1 nos dicen
- Los 5 scores son muy altos, lo que quiere decir que el pipeline esta consistentemente obteniendo un score alto en "nueva" informacion. El pipeline generaliza muy bien cada 20% del dataset
- Este modelo representa una mejora ante los dos modelos implementados previamente
  - Regresion Logistica (sin "score_calidad"): media CV macro-F1 ≈ 0.909
  - Random Forest (sin "score_calidad"): media CV macro-F1 ≈ 0.8613
- Hasta este punto el modelo de LightGBM tiene el mejor rendimiento categorizando el dataset y prediciendo "calidad_auto"

#### LightGBM testing

In [143]:
# Fit LGBM pipeline
lgbm_pipeline.fit(X_train, y_train)

# Predecir en el split de 20%
y_lgbm_pred = lgbm_pipeline.predict(X_test)

# Evaluar el modelo
lgbm_test_acc = accuracy_score(y_test, y_lgbm_pred)
print(f"\nLightGBM Test Accuracy: {lgbm_test_acc:.4f}")

# Classification report
print("\nLightGBM Report (Test Set):\n")
print(classification_report(
    y_test,
    y_lgbm_pred,
    digits=4,
    zero_division=0
))

# Confusion matrix
lgbm_cm = confusion_matrix(y_test, y_lgbm_pred, labels=["Media", "Alta", "Baja"])
print("\nRandom Forest Confusion Matrix (Test Set):\n", lgbm_cm)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1372
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 27
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612

LightGBM Test Accuracy: 0.9755

LightGBM Report (Test Set):

              precision    recall  f1-score   support

        Alta     0.9302    0.9524    0.9412       210
        Baja     0.8583    0.9364    0.8957       110
       Media     0.9898    0.9810    0.9854      1680

    accuracy                         0.9755      2000
   macro avg     0.9261    0.9566    0.9407      2000
weighted avg     0.9763    0.9755    0.9758      2000


Random Forest Confusion Matrix (Test Set):

### Evaluation